# System Identification of a Freely Falling Ball

The data in `data/falling_object.npy` provides the motion of a ball (only its position), freely falling (without friction) under the gravity of one of the solar system's planets. You are supposed to find which planet the ball is closest to. Perform this task in two different ways:

1. Implement the adjoint differentiation method in Python/JAX and use the gradients provided by this method to figure out the gravitational constant.
2. Use JAX 's built-in functions `jax.grad` or `jax.value_and_grad` to replace the adjoint differentiation method and repeat the task.
3. Do both methods converge? Which method is faster? Can you make JAX 's grad (or value_and_grad) work faster?

The data contains 1000 floats that correspond to the vertical motion of the ball. Each time step corresponds to 0.001 seconds of motion. The time interval between the initial and final time step is thus 1 second.

In [1]:
from data import fallingobject

# Load height trajectory of the object.
ut, t_eval, dt = fallingobject.load_data()

## Problem Definition

The following second-order ODE represents the system behavior.
$$
    \frac{d^2x}{dt^2} = \ddot x = g,            \tag{1}
$$
where $x$ is the height of the object and $g$ is the unknown gravitational acceleration.
An analytical solution $x(t)$ of this simple ODE follows from double integration:
$$
    v(t) = \int_{0}^{t} g d\tau = v(0) + g t,   \tag{2}
$$$$
    x(t) = \int_{0}^{t} v(\tau) d\tau = x(0) + v(0) t + \frac{1}{2} g t^2.  \tag{3}
$$
Given the measurements, we know the initial position $x(0)$, but the initial velocity $v(0)$ and $g$ are unknown.
Since Eq. (3) is linear in the unknown parameters, we can find the least-square estimate of $v(0)$ and $g$, but the solution will serve as **ground truth values** for the subsequent analyses.

In [2]:
import numpy as np
# Coefficient matrix of the unknown vector (v0, g) and the RHS of eq.
A = np.array([[t, 0.5*t*t] for t in t_eval])
b = np.array(ut - ut[0]).reshape(-1)
# Least squared estimate of the unknowns.
x, _, _, _ = np.linalg.lstsq(A, b, rcond=None)
g_best = x[1]
v0_best = x[0]
print(f"LS estimate of grav. acc.: {g_best:.5f}")
print(f"LS estimate of init. vel.: {v0_best:.5f}")

LS estimate of grav. acc.: -3.70002
LS estimate of init. vel.: 0.10185


## System Model

It is convenient to model this simple problem as a system of first-order ODEs to explore the methodology before applying it to more complex scenarios.
$$
    \frac{d\mathbf{u}}{dt}(t) = \mathbf{f}(\mathbf{u}(t), t; \mathbf{p})    \tag{4}
    =
    \begin{bmatrix}
        f_1(\mathbf{u}(t), t; \mathbf{p}) \\
        f_2(\mathbf{u}(t), t; \mathbf{p})
    \end{bmatrix}
    =
    \begin{bmatrix} u_2 \\ p_1 \end{bmatrix},\;\;\;
    \mathbf{u}(t_0) = \begin{bmatrix} u_1(t_0) \\ p_2 \end{bmatrix},
$$
where the variables $u_1 = x$, $u_2 = \dot x = v$, and the parameter $p_1 = g$.
This form of the problem also inherits the **unknown initial velocity** $p_2 = v(t_0)$ as the previous approach.
Therefore, the following section considers optimizing these two parameters using the adjoint method.

## Optimization of the Unknown Parameters

We can find the optimal parametrization of the system model in Eq. (2), complying with the measurements $u_1(t)$ for some time range, by defining an objective (loss) function of the form:
$$
\begin{align}
    J(\mathbf{u}(t); \mathbf{p}) &=
        \int_{t_0}^{T} h(\mathbf{u}(t); \mathbf{p}) dt,                     \tag{5} \\
    h(\mathbf{u}(t); \mathbf{p}) &=
        \frac{1}{2} (u_1(t; \mathbf{p}) - u_1^{*}(t))^2,                    \tag{6}
\end{align}
$$
where $u_1^{*}$ are the true values.
The parameters with the least average error along the measured trajectory are obtained by minimizing Eq. (5) over the parameter(s) $\mathbf{p}$ subject to Eq. (4).
However, the constraint optimization can be solved by minimizing the Lagrangian function.
$$
    \mathcal{L}(\mathbf{u}(t); \mathbf{p}) =                                \tag{7}
    J(\mathbf{u}(t); \mathbf{p}) + \int_{t_0}^{T} \lambda(t)^\top \left(
        \mathbf{f}(\mathbf{u}(t), t; \mathbf{p}) - \frac{d\mathbf{u}}{dt}(t) \right) dt,
$$
where the expression in parentheses is zero for all $t$, giving a freedom in choosing $\lambda(t)$.
After applying integration by parts and rearrangement of the terms, the gradient of the objective becomes:
$$
\begin{align}
    \frac{d\mathcal{L}}{d\mathbf{p}} = \frac{dJ}{d\mathbf{p}} =& \int_{t_0}^{T} \left[
        \frac{\delta h}{\delta \mathbf{p}}
        + \lambda(t)^{\top} \frac{\delta \mathbf{f}}{\delta \mathbf{p}}
        + \left( \frac{\delta h}{\delta \mathbf{u}}
            + \lambda(t)^{\top} \frac{\delta \mathbf{f}}{\delta \mathbf{u}}
            + \frac{d \lambda(t)^{\top}}{dt}
        \right) \frac{\delta \mathbf{u}}{\delta \mathbf{p}}
    \right] dt \nonumber \\
    &+ \lambda(t_0)^{\top} \frac{\delta \mathbf{u}}{\delta \mathbf{p}}(t_0)
    - \lambda(T)^{\top} \frac{\delta \mathbf{u}}{\delta \mathbf{p}}(T).     \tag{8}
\end{align}
$$
Since we are free to choose $\lambda(t)$, we can form the following adjoint ODE (as a terminal-value problem),
$$
    \frac{d \lambda(t)}{dt} =                                               \tag{9}
        - \frac{\delta \mathbf{f}}{\delta \mathbf{u}}^{\top} \lambda(t)
        - \frac{\delta h}{\delta \mathbf{u}}^{\top},
    \;\; \lambda(T) = 0,
$$
so that we can eliminate the difficult-to-calculate $\frac{\delta u}{\delta \mathbf{p}}$ term.
Then, the sensitivity $\frac{d\mathcal{L}}{dp}$ to be used in gradient descent can be calculated efficiently as follows.

- Numerically solve the original ODE (initial-value problem) given by Eq. (4) forward to estimate the solution $\hat{\mathbf{u}}(t)$.
- Numerically solve the adjoint terminal-value problem given by Eq. (9) backward to estimate the solution $\lambda(t)$.
- Numerically evaluate the integral in Eq. (10) forward (by setting $w(0) = 0$) or backward (by setting $w(T) = 0$). The result of the evaluation is the value of the $\frac{d\mathcal{L}}{d\mathbf{p}}$ or its negative based on the integration direction.

$$
    \frac{dJ}{d\mathbf{p}} = \int_{t_0}^{T} \left[                          \tag{10}
        \frac{\delta h}{\delta \mathbf{p}}
         + \lambda(t)^{\top} \frac{\delta \mathbf{f}}{\delta \mathbf{p}}
    \right] dt + \lambda(t_0)^{\top} \frac{\delta \mathbf{u}}{\delta \mathbf{p}}(t_0).
$$

Then, the update rule of the gradient descent algorithm is
$$
    \mathbf{p} \leftarrow \mathbf{p} - \eta \frac{dJ}{d\mathbf{p}}.         \tag{11}
$$

In the subsequent sections, we attempt to solve the problem by comparing the performances of different tools.

In [3]:
# Common gradient descent settings:
num_epochs = 2000
eta = 2     # Learning rate of the gradient descent.
g0 = -9.81  # Initial guess for the gravity: assume Earth.
v0 = 0      # Initial guess for the velocity: assume zero.

## Analytical Differentiation of the Terms

The procedure described in previous section requires calculation of several partial derivatives. In this section, we use the analytical differentiation.
$$
    \frac{\delta \mathbf{f}}{\delta \mathbf{u}} =
        \begin{bmatrix} 0 & 1 \\ 0 & 0 \end{bmatrix}, \;\;\;\;\;
    \frac{\delta h}{\delta \mathbf{u}} =
        \begin{bmatrix} u_1(t) - u_1^*(t) & 0 \end{bmatrix}                         \tag{12}
$$$$
    \frac{\delta \mathbf{f}}{\delta \mathbf{p}} =
        \begin{bmatrix} 0 & 0 \\ 1 & 0 \end{bmatrix}, \;\;\;\;\;
    \frac{\delta h}{\delta \mathbf{p}} =
        \begin{bmatrix} 0 & 0 \end{bmatrix}, \;\;\;\;\;
    \frac{\delta \mathbf{u}}{\delta \mathbf{p}}(t_0) =
        \begin{bmatrix} 0 & 0 \\ 0 & 1 \end{bmatrix}                                \tag{13}
$$
Plugging Eq. (12) into (9) and Eq. (13) into (10), we can follow the procedure to find the parameters as follows.

In [4]:
from odes import falling_ball_adjoint

pred_gv, u_sols, costs = falling_ball_adjoint.optimize(
    v0, g0, ut, t_eval, eta, num_epochs
)
print(f"Grav. acc.: Predicted ({pred_gv[-1, 0]:.5f})"
      f" ~= ({g_best:.5f}) LS estimation")
print(f"Init. vel.: Predicted ( {pred_gv[-1, 1]:.5f})"
      f" ~= ( {v0_best:.5f}) LS estimation")

Grav. acc.: Predicted (-3.70011) ~= (-3.70002) LS estimation
Init. vel.: Predicted ( 0.10189) ~= ( 0.10185) LS estimation
